In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 
import cv2
from PIL import Image

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [121]:
file_root = "E:\Satellite image segmentation\dataset"
df = pd.read_csv("E:\Satellite image segmentation\dataset\metadata.csv")
df_train = df.loc[df['split']=='train']
df_test = df.loc[df['split']== 'test']
for i in range(0,len(df_train)):
    df_train['png_image_path']=df_train['png_image_path'][i].replace("/","\\")
    df_train['png_label_path']=df_train['png_label_path'][i].replace("/","\\")
for i in range(df_test.index[0],df_test.index[-1]):
    df_test['png_image_path']=df_test['png_image_path'][i].replace("/","\\")
    df_test['png_label_path']=df_test['png_label_path'][i].replace("/","\\")

df_train

C:\Users\shuga\AppData\Local\Temp\ipykernel_24612\1446968779.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['png_image_path']=df_train['png_image_path'][i].replace("/","\\")
C:\Users\shuga\AppData\Local\Temp\ipykernel_24612\1446968779.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['png_label_path']=df_train['png_label_path'][i].replace("/","\\")
C:\Users\shuga\AppData\Local\Temp\ipykernel_24612\1446968779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,image_id,split,image_souce_url,label_source_url,tiff_image_path,tif_label_path,png_image_path,png_label_path
0,22678915_15,train,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,tiff/train/22678915_15.tiff,tiff/train_labels/22678915_15.tif,png\train\22678915_15.png,png\train_labels\22678915_15.png
1,22678930_15,train,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,tiff/train/22678930_15.tiff,tiff/train_labels/22678930_15.tif,png\train\22678915_15.png,png\train_labels\22678915_15.png
2,22678945_15,train,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,tiff/train/22678945_15.tiff,tiff/train_labels/22678945_15.tif,png\train\22678915_15.png,png\train_labels\22678915_15.png
3,22678960_15,train,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,tiff/train/22678960_15.tiff,tiff/train_labels/22678960_15.tif,png\train\22678915_15.png,png\train_labels\22678915_15.png
4,22678975_15,train,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,tiff/train/22678975_15.tiff,tiff/train_labels/22678975_15.tif,png\train\22678915_15.png,png\train_labels\22678915_15.png
...,...,...,...,...,...,...,...,...
132,24478855_15,train,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,tiff/train/24478855_15.tiff,tiff/train_labels/24478855_15.tif,png\train\22678915_15.png,png\train_labels\22678915_15.png
133,24478870_15,train,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,tiff/train/24478870_15.tiff,tiff/train_labels/24478870_15.tif,png\train\22678915_15.png,png\train_labels\22678915_15.png
134,24478885_15,train,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,tiff/train/24478885_15.tiff,tiff/train_labels/24478885_15.tif,png\train\22678915_15.png,png\train_labels\22678915_15.png
135,24478900_15,train,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,http://www.cs.toronto.edu/~vmnih/data/mass_bui...,tiff/train/24478900_15.tiff,tiff/train_labels/24478900_15.tif,png\train\22678915_15.png,png\train_labels\22678915_15.png


In [146]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # imagenet
                         [0.229, 0.224, 0.225])
])

class Dataset(Dataset):
    def __init__(self,df,root=file_root,transforms=None):
        super(Dataset).__init__()
        self.df = df
        self.root = file_root 
        self.transform = transforms
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.root+"\\"+self.df['png_image_path'][idx]
        mask_path = self.root+'\\'+self.df['png_label_path'][idx]
        img = Image.open(img_path)
        mask =  Image.open(mask_path)
        if self.transform is not None:
            img  = self.transform(img)
            mask = self.transform(mask)
        
        return img,mask


        
train_dataset = Dataset(df=df_train,root=file_root,transforms=transform)
test_dataset = Dataset(df=df_test,root=file_root,transforms=transform)
train_loader = DataLoader(train_dataset,batch_size=4,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=1,shuffle=False)

150